In [1]:
import numpy as np


In [2]:
class TreeNode:
    def __init__(self, depth=0):
        self.left = None
        self.right = None
        self.feature = None
        self.threshold = None
        self.value = None
        self.depth = depth

class XGBoostClassifierScratch:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, 
                 min_child_weight=1, gamma=0, reg_lambda=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.gamma = gamma
        self.reg_lambda = reg_lambda
        self.trees = []
        self.F0 = None  # initial log-odds

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _calc_grad_hess(self, y, pred):
        p = self._sigmoid(pred)
        grad = p - y
        hess = p * (1 - p)
        return grad, hess

    def _calc_leaf_weight(self, grad, hess):
        return -np.sum(grad) / (np.sum(hess) + self.reg_lambda)

    def _calc_gain(self, G_L, H_L, G_R, H_R):
        gain = 0.5 * (G_L**2 / (H_L + self.reg_lambda) + G_R**2 / (H_R + self.reg_lambda) -
                      (G_L + G_R)**2 / (H_L + H_R + self.reg_lambda)) - self.gamma
        return gain

    def _build_tree(self, X, grad, hess, depth=0):
        node = TreeNode(depth=depth)
        n_samples, n_features = X.shape

        # Stop conditions
        if depth >= self.max_depth or n_samples <= 1:
            node.value = self._calc_leaf_weight(grad, hess)
            return node

        best_gain = -np.inf
        best_feature = None
        best_thresh = None

        # Try all features and thresholds
        for f in range(n_features):
            thresholds = np.unique(X[:, f])
            for t in thresholds:
                left_mask = X[:, f] <= t
                right_mask = ~left_mask

                if np.sum(left_mask) < 1 or np.sum(right_mask) < 1:
                    continue

                G_L, H_L = np.sum(grad[left_mask]), np.sum(hess[left_mask])
                G_R, H_R = np.sum(grad[right_mask]), np.sum(hess[right_mask])

                if H_L < self.min_child_weight or H_R < self.min_child_weight:
                    continue

                gain = self._calc_gain(G_L, H_L, G_R, H_R)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = f
                    best_thresh = t

        if best_gain <= 0:
            node.value = self._calc_leaf_weight(grad, hess)
            return node

        # Otherwise, split
        node.feature = best_feature
        node.threshold = best_thresh
        left_mask = X[:, best_feature] <= best_thresh
        right_mask = ~left_mask
        node.left = self._build_tree(X[left_mask], grad[left_mask], hess[left_mask], depth + 1)
        node.right = self._build_tree(X[right_mask], grad[right_mask], hess[right_mask], depth + 1)
        return node

    def _predict_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict_tree(x, node.left)
        else:
            return self._predict_tree(x, node.right)

    def fit(self, X, y):
        # Initialize F0 with log-odds
        pos_ratio = np.clip(np.mean(y), 1e-5, 1-1e-5)
        self.F0 = np.log(pos_ratio / (1 - pos_ratio))
        pred = np.full(y.shape, self.F0)

        for _ in range(self.n_estimators):
            grad, hess = self._calc_grad_hess(y, pred)
            tree = self._build_tree(X, grad, hess)
            self.trees.append(tree)
            pred += self.learning_rate * np.array([self._predict_tree(x, tree) for x in X])

    def predict_proba(self, X):
        pred = np.full(X.shape[0], self.F0)
        for tree in self.trees:
            pred += self.learning_rate * np.array([self._predict_tree(x, tree) for x in X])
        p = self._sigmoid(pred)
        return np.vstack([1-p, p]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)


In [3]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

X, y = make_classification(n_samples=200, n_features=5, n_informative=3, n_redundant=0, random_state=42)
xgb_clf = XGBoostClassifierScratch(n_estimators=50, learning_rate=0.1, max_depth=3, gamma=0.1)
xgb_clf.fit(X, y)
y_pred = xgb_clf.predict(X)
print("Scratch XGBoost Accuracy:", accuracy_score(y, y_pred))


Scratch XGBoost Accuracy: 0.98


In [ ]:
from xgboost import XGBClassifier
sklearn_clf = XGBClassifier(n_estimators=50, learning_rate=0.1, max_depth=3, use_label_encoder=False, verbosity=0)
sklearn_clf.fit(X, y)
y_pred_sklearn_clf = sklearn_clf.predict(X)